In [1]:
import pandas as pd
import numpy as np

In [9]:
scores=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Scores 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')
order_states=pd.read_csv('/rds/general/user/hsl121/home/hda_project/data/2025 AI Imperial Dataset Orders_Prescription_Formulas.csv')
prescription_formula=pd.read_csv('/rds/general/user/hsl121/home/hda_project/data/2025 AI Imperial Dataset Orders_Prescription_Formulas.csv')
adverse=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Adverse Events 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')
comorbidites=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Comorbidities 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')
medications=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Medications 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')
prescription=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Prescriptions 6 Jan 2025_Prescribed_Completed Baseline PROMs NEW.xlsx')

In [45]:
eq5d=scores[scores['promName']=='EQ5D']
eq5d=eq5d[['SID','Round','promName','total_score']]

In [15]:
print(eq5d.columns)

Index(['SID', 'Round', 'promName', 'total_score'], dtype='object')


In [16]:
xls = pd.ExcelFile('/rds/general/user/hsl121/home/hda_project/Protocol and data framework/Copy of EQ-5D-5L_Crosswalk_Index_Value_Calculator_MAC.xls')
print(xls.sheet_names)


['Title', 'Index Value Calculator', 'EQ-5D-5L Value Sets']


In [42]:
eq5d_index = pd.read_excel(xls, sheet_name='EQ-5D-5L Value Sets', skiprows=1)


In [43]:
eq5d_index = eq5d_index.drop(eq5d_index.columns[0], axis=1)
eq5d_index= eq5d_index[['5L profile', 'UK']]


In [44]:
eq5d_index.columns

Index(['5L profile', 'UK'], dtype='object')

In [47]:
eq5d.columns

Index(['SID', 'Round', 'promName', 'total_score'], dtype='object')

In [48]:
eq5d = eq5d.merge(eq5d_index, left_on='total_score', right_on='5L profile', how='left')


In [50]:
eq5d.head

<bound method NDFrame.head of                                      SID  Round promName  total_score  \
0           003adIEhRDbcWiL9YN4rjAvekGl1      1     EQ5D      43455.0   
1           003adIEhRDbcWiL9YN4rjAvekGl1      2     EQ5D      31222.0   
2           003adIEhRDbcWiL9YN4rjAvekGl1      3     EQ5D      33222.0   
3       00b622bb2e064de91714bb4d7812245e      1     EQ5D      14424.0   
4       00b622bb2e064de91714bb4d7812245e      2     EQ5D      24434.0   
...                                  ...    ...      ...          ...   
113594              zZwQ0lYo31weDN038pOq      2     EQ5D      11222.0   
113595              zZwQ0lYo31weDN038pOq      3     EQ5D      11122.0   
113596      zZXvDhEI5neNrU419i5tlcvlEkz1      1     EQ5D      32544.0   
113597      zZXvDhEI5neNrU419i5tlcvlEkz1      2     EQ5D      21222.0   
113598      zzZ75pcuChS4zFv3wX4TchiEcVt2      1     EQ5D      32344.0   

        5L profile     UK  
0          43455.0 -0.218  
1          31222.0  0.666  
2        

In [46]:

eq5d_wide = eq5d.pivot_table(index='SID', 
                                    columns='Round', 
                                    values='total_score', 
                                    aggfunc='first')

eq5d_wide.columns = [f"EQ5D_Round{r}" for r in eq5d_wide.columns]
eq5d_wide = eq5d_wide.reset_index()


In [13]:
eq5d_wide.columns

Index(['SID', 'EQ5D_Round1', 'EQ5D_Round2', 'EQ5D_Round3', 'EQ5D_Round4',
       'EQ5D_Round5', 'EQ5D_Round6', 'EQ5D_Round7', 'EQ5D_Round8',
       'EQ5D_Round9', 'EQ5D_Round10', 'EQ5D_Round11', 'EQ5D_Round12',
       'EQ5D_Round13'],
      dtype='object')

In [31]:
eq5d_df=pd.merge(eq5d_wide, comorbidites, on='SID', how='left')

In [39]:
print(eq5d_df['EQ5D_Round13'].isnull().sum())

31432


In [24]:
print(scores['SID'].nunique())

31508


RQ1 Dataset

In [4]:

proms_to_keep = ['GAD7', 'EQ5D', 'PGIC', 'insomniaEfficacyMeasure']
scores_filtered = scores[scores['promName'].isin(proms_to_keep)]

scores_filtered['Round'] = scores_filtered['Round'].astype(str)
scores_filtered['PROM_Round'] = scores_filtered['promName'] + '_Round' + scores_filtered['Round']

# Pivot to wide format
scores_wide = scores_filtered.pivot_table(index='SID', 
                                          columns='PROM_Round', 
                                          values='total_score', 
                                          aggfunc='mean')

# Reset index
scores_wide = scores_wide.reset_index()


/rds/general/ephemeral/user/hsl121/ephemeral/ipykernel_3276417/542726840.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_filtered['Round'] = scores_filtered['Round'].astype(str)
/rds/general/ephemeral/user/hsl121/ephemeral/ipykernel_3276417/542726840.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_filtered['PROM_Round'] = scores_filtered['promName'] + '_Round' + scores_filtered['Round']


['Title', 'Index Value Calculator', 'EQ-5D-5L Value Sets']


In [8]:
eq5d_index.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Denmark', 'France', 'Germany', 'Japan',
       'Netherlands', 'Spain', 'Thailand', 'UK', 'US', 'Zimbabwe'],
      dtype='object')

In [5]:
prescription['DateofPrescription'] = pd.to_datetime(prescription['DateofPrescription'])

first_prescription = (
    prescription.sort_values(by='DateofPrescription')
    .drop_duplicates(subset='SID', keep='first')
    .reset_index(drop=True)
)

first_prescription = first_prescription[['SID', 'DateofPrescription', 'Product','Totalquantity','Route']]
first_prescription.columns = ['SID', 'FirstPrescriptionDate', 'FirstProduct','FirstTotalQuantity','Route']

In [73]:
first_prescription.head


<bound method NDFrame.head of                                     SID FirstPrescriptionDate  \
0      d4f47ff0ce92f9e63dc727192aab8ee5            2019-12-01   
1                  jeqr4rp3lX5uyZIo9hGR            2020-08-07   
2      f706a33d2c41c6c99429921310f26ef3            2020-08-07   
3                  USAXrNl8qJsWNMrXd7eK            2020-08-18   
4      ec4150acb39aee85e967c7649b9902ee            2020-08-18   
...                                 ...                   ...   
31503      D7XHnB4UQXS4iHuf2uXXgw4ykTq1            2025-01-02   
31504      QbfdIHi68KQJWkVZ7Sms4rcSAy83            2025-01-03   
31505      LMrsayxBMCcDnxDKss1DsGGnfoO2            2025-01-03   
31506      UZ24zeHWSZfQsxbS0RVERJZ5nBo1            2025-01-03   
31507      zN7qilOyfYNvZdUbkKOGD8zFFbZ2            2025-01-03   

                                            FirstProduct FirstTotalQuantity  \
0                                       Tilray 10:10 oil     25 millilitres   
1                           Bed

In [6]:
rq1=pd.merge(scores_wide, first_prescription, on='SID', how='left')


In [7]:
rq1=pd.merge(rq1, comorbidites, on='SID', how='outer')

In [76]:
rq1.columns

Index(['SID', 'EQ5D_Round1', 'EQ5D_Round10', 'EQ5D_Round11', 'EQ5D_Round12',
       'EQ5D_Round13', 'EQ5D_Round2', 'EQ5D_Round3', 'EQ5D_Round4',
       'EQ5D_Round5',
       ...
       'Cannabis_use_frequency', 'Cannabis_use_route', 'Cannabis_gram_per_day',
       'Cannabis_gram_years', 'Prescribed', 'BaselinePROMS',
       'FirstPrescriptionDate_y', 'FirstYearPrescribed', 'Status',
       'GovernmentOfficeRegion'],
      dtype='object', length=104)

In [8]:
drop=['Cannabis_gram_per_day', 'Prescribed', 'BaselinePROMS','FirstPrescriptionDate_y', 'FirstYearPrescribed', 'Status','GovernmentOfficeRegion']

rq1=rq1.drop(drop, axis=1, inplace=False)

In [78]:
rq1.columns

Index(['SID', 'EQ5D_Round1', 'EQ5D_Round10', 'EQ5D_Round11', 'EQ5D_Round12',
       'EQ5D_Round13', 'EQ5D_Round2', 'EQ5D_Round3', 'EQ5D_Round4',
       'EQ5D_Round5', 'EQ5D_Round6', 'EQ5D_Round7', 'EQ5D_Round8',
       'EQ5D_Round9', 'GAD7_Round1', 'GAD7_Round10', 'GAD7_Round11',
       'GAD7_Round12', 'GAD7_Round13', 'GAD7_Round2', 'GAD7_Round3',
       'GAD7_Round4', 'GAD7_Round5', 'GAD7_Round6', 'GAD7_Round7',
       'GAD7_Round8', 'GAD7_Round9', 'PGIC_Round1', 'PGIC_Round10',
       'PGIC_Round11', 'PGIC_Round12', 'PGIC_Round13', 'PGIC_Round2',
       'PGIC_Round3', 'PGIC_Round4', 'PGIC_Round5', 'PGIC_Round6',
       'PGIC_Round7', 'PGIC_Round8', 'PGIC_Round9',
       'insomniaEfficacyMeasure_Round1', 'insomniaEfficacyMeasure_Round10',
       'insomniaEfficacyMeasure_Round11', 'insomniaEfficacyMeasure_Round12',
       'insomniaEfficacyMeasure_Round13', 'insomniaEfficacyMeasure_Round2',
       'insomniaEfficacyMeasure_Round3', 'insomniaEfficacyMeasure_Round4',
       'insomniaEffica

In [58]:
print(rq1.columns.values)

['SID' 'AKPS_Round1' 'AKPS_Round10' 'AKPS_Round11' 'AKPS_Round2'
 'AKPS_Round3' 'AKPS_Round4' 'AKPS_Round5' 'AKPS_Round6' 'AKPS_Round7'
 'AKPS_Round8' 'AKPS_Round9' 'EDEQ_Round1' 'EDEQ_Round10' 'EDEQ_Round2'
 'EDEQ_Round3' 'EDEQ_Round4' 'EDEQ_Round5' 'EDEQ_Round6' 'EDEQ_Round7'
 'EDEQ_Round8' 'EDEQ_Round9' 'EQ5D_Round1' 'EQ5D_Round10' 'EQ5D_Round11'
 'EQ5D_Round12' 'EQ5D_Round13' 'EQ5D_Round2' 'EQ5D_Round3' 'EQ5D_Round4'
 'EQ5D_Round5' 'EQ5D_Round6' 'EQ5D_Round7' 'EQ5D_Round8' 'EQ5D_Round9'
 'GAD7_Round1' 'GAD7_Round10' 'GAD7_Round11' 'GAD7_Round12' 'GAD7_Round13'
 'GAD7_Round2' 'GAD7_Round3' 'GAD7_Round4' 'GAD7_Round5' 'GAD7_Round6'
 'GAD7_Round7' 'GAD7_Round8' 'GAD7_Round9' 'HIT_Round1' 'HIT_Round10'
 'HIT_Round11' 'HIT_Round2' 'HIT_Round3' 'HIT_Round4' 'HIT_Round5'
 'HIT_Round6' 'HIT_Round7' 'HIT_Round8' 'HIT_Round9' 'IBDQ_Round1'
 'IBDQ_Round10' 'IBDQ_Round11' 'IBDQ_Round2' 'IBDQ_Round3' 'IBDQ_Round4'
 'IBDQ_Round5' 'IBDQ_Round6' 'IBDQ_Round7' 'IBDQ_Round8' 'IBDQ_Round9'
 'IEPS_Rou

In [ ]:
# Ensure both date columns are in datetime format
scores['completionDate'] = pd.to_datetime(scores['completionDate'], errors='coerce')
prescription['FirstPrescriptionDate'] = pd.to_datetime(prescription['FirstPrescriptionDate'], errors='coerce')

# Filter scores to only Round 1 (baseline)
baseline_scores = scores[scores['Round'].astype(str) == '1'].copy()

# Merge in the prescription date
baseline_scores = pd.merge(baseline_scores, 
                           prescription[['SID', 'FirstPrescriptionDate']], 
                           on='SID', how='left')

# Filter: keep only if the PROM baseline was on or after the prescription date
baseline_scores = baseline_scores[baseline_scores['completionDate'] >= baseline_scores['FirstPrescriptionDate']]


KeyError: 'FirstPrescriptionDate'